# MSSQL Integration Demo

### Imports & Setup

In [17]:
%load_ext autoreload
%autoreload 2
from datetime import datetime
from zoneinfo import ZoneInfo
from helpers.irp_integration import IRPClient
from helpers.sqlserver import execute_query_from_file, execute_script_file

irp_client = IRPClient()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Setup

In [ ]:
connection_name = "DATABRIDGE"
peril = "USEQ"
edm_name = f"{peril}_Automated"
portfolio_name = f"{peril}_Automated"
date_value = "202511"

### Portfolio Mapping

In [19]:
edms = irp_client.edm.search_edms(filter=f"exposureName=\"{edm_name}\"")
if (len(edms) != 1):
    raise Exception(f"Found {len(edms)} EDMs with name {edm_name}; expected 1")
edm = edms[0]
exposure_id = edm['exposureId']
edm_full_name = edm['databaseName']
print(f'EDM Full Name: {edm_full_name}')

portfolios = irp_client.portfolio.search_portfolios(exposure_id, f"portfolioName=\"{portfolio_name}\"")
if (len(portfolios) != 1):
    raise Exception(f"Found {len(edms)} Portfolios with name {portfolio_name}; expected 1")
portfolio_id = portfolios[0]['portfolioId']
print(f'Portfolio ID: {portfolio_id}')
print()

sql_script_name = f"2b_Query_To_Create_Sub_Portfolios_{peril}_RMS_BackEnd.sql"
params = {
    "EDM_FULL_NAME": edm_full_name,
    "PORTFOLIO_ID": portfolio_id,
    "DATETIME_VALUE": datetime.now(ZoneInfo("America/New_York")).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
}

result = execute_script_file(
    file_path = f'portfolio_mapping/{sql_script_name}',
    params = params,
    connection = connection_name
)

print(result)

EDM Full Name: USEQ_Automated_rGLG
Portfolio ID: 1
Reading SQL script ...
Parameterizing ...
Executing SQL ...
Script Execution Result: ✓ SUCCESS
  Statements executed: 1
  Rows affected:       0
  Result sets:         0


### Control Totals

In [20]:
workspace_edm = "WORKSPACE_EDM"
edms = irp_client.edm.search_edms(filter=f"exposureName=\"{edm_name}\"")
if (len(edms) != 1):
    raise Exception(f"Found {len(edms)} EDMs with name {edm_name}; expected 1")
edm = edms[0]
edm_full_name = edm['databaseName']
print(f'Workspace EDM Full Name: {edm_full_name}')
print()

sql_script_name = '3d_RMS_EDM_CONTROL_Totals.sql'
params = {
    "WORKSPACE_EDM": edm_full_name,
    "CYCLE_TYPE": "Quarterly",
    "DATETIME_VALUE": datetime.now(ZoneInfo("America/New_York")).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
}

result = execute_script_file(
    file_path = f'control_totals/{sql_script_name}',
    params = params,
    connection = connection_name
)

print(result)

Workspace EDM Full Name: USEQ_Automated_rGLG

Reading SQL script ...
Parameterizing ...


SQLServerQueryError: Script execution failed (connection: DATABRIDGE, file: control_totals/3d_RMS_EDM_CONTROL_Totals.sql): Missing required parameter: 'DATE_VALUE'
Query requires parameter that was not provided.
Provided parameters: WORKSPACE_EDM, CYCLE_TYPE, DATETIME_VALUE

### Execute Simple SQL Scripts

In [ ]:
connection_name = "DATABRIDGE"
sql_script_name = "list_edm_tables.sql"

dataframes = execute_query_from_file(
    f'examples/{sql_script_name}',
    params={'EDM_FULL_NAME': edm_full_name},
    connection=connection_name
)

for df in dataframes:
    print(f"Query returned {len(df)} rows")
    print("\nResults:")
    print(df)

In [ ]:
connection_name = "DATABRIDGE"
sql_script_name = "test_multiple_select.sql"

dataframes = execute_query_from_file(
    f'examples/{sql_script_name}',
    params={'EDM_FULL_NAME': edm_full_name},
    connection=connection_name
)

for df in dataframes:
    print(f"Query returned {len(df)} rows")
    print("\nResults:")
    print(df.head())